In [1]:
from tkinter import ttk
from tkinter import *
from tkinter.scrolledtext import ScrolledText
from smartcard.System import readers
from smartcard.Exceptions import (CardConnectionException,NoCardException, SmartcardException)
from smartcard.util import toHexString
import time, threading
from tkinter import messagebox
from tkinter import simpledialog



def gui():
    root = Tk()
    root.title("Smart Card Reader SLE4442 and SLE4428")

    nb = ttk.Notebook(root)

    # adding Frames as pages for the ttk.Notebook 
    # first page, which would get widgets gridded into it
    commands = Frame(nb,width=800, height=600)
    
    text = ScrolledText(commands)
    text.pack(expand=1, fill="both")
    # second page
    card_root = Frame(nb,  width=800, height=600)
    
    # second page
    atr = Frame(nb,  width=800, height=600)


    nb.add(commands, text='Commands')
    nb.add(card_root, text='Card')
    nb.add(atr, text='ATR')

    nb.pack(expand=1, fill="both")
    atrLb=Label(atr, text="Atr: ")
    atrLb.grid(row=0,column=0, padx=2, pady=2,sticky='ne')
    
    card_head = Frame(card_root,  width=800, height=600)
    card_head.grid(row=0, column=0, padx=5, pady=5)
    global r,connection,select_card,auth
    
    auth=False
    r=readers()
    def addCommands(cm_type,command):
        
        text.insert(END,"\n"+ cm_type)
        text.insert(END, command)
    
    selectedCard = StringVar(root)
    # Dictionary with options
    choices = { 'SLE 4442','SLE 4428'}
    selectedCard.set('SLE 4442') # set the default option

    popupMenu = OptionMenu(card_head, selectedCard, *choices)
    Label(card_head, text="Choose a card").grid(row = 0, column = 0)
    popupMenu.grid(row = 0, column =1)

    # on change dropdown value
    def change_selection(*args):
        global select_card
        print( selectedCard.get() )
        if selectedCard.get()=="SLE 4442":
            select_card=0x06
        elif selectedCard.get()=="SLE 4428":
            select_card=0x05

    # link function to change dropdown
    selectedCard.trace('w', change_selection)
    select_card=0x06
    def Connect():
    
        global r,connection,atr,select_card,auth
        
        auth=False
        r=readers()
        autLb.config(fg="red",text="Unauthorized ")
        if len(r)==0:
            connectLb.config(fg="red",text="Do not find card reader.")
        else:
            
            addCommands("Reader ",r)
            
            try:
                connection = r[0].createConnection()
                connection.connect()
                atr=toHexString(connection.getATR() )
                addCommands("ATR: ",atr)
                atrLb.config(text="Atr: "+atr)
                
                # SELECT_CARD_TYPE
                SELECT = [0xFF, 0xA4, 0x00, 0x00,0x01,select_card]
                
                addCommands("SELECT_CARD_TYPE ",toHexString(SELECT))
                data, sw1, sw2 = connection.transmit(SELECT)
                
                addCommands("Respons: ","["+toHexString(data)+"] "+str(hex(sw1))+" "+str(hex(sw2)))
                
                connectLb.config(fg="green",text="Connected ")
                ReadMainMemory()
                
            except:                
                connectLb.config(fg="red",text="Please insert smart card. ")
                addCommands("Error ","Please insert smart card. ")
                pass

    btn_connect = Button(card_head, text ="Connect", command = Connect, padx=5, pady=2)
    btn_connect.grid(row=0,column=2,sticky='ne')
    
    connectLb=Label(card_head, text="Please connect ",fg="red")
    connectLb.grid(row=0,column=3, padx=2, pady=2,sticky='ne')
    
    card = Frame(card_root,  width=800, height=600)
    card.grid(row=1, column=0, padx=5, pady=5)
    
    Label(card, text="Security Memory ").grid(row=0,column=0, padx=2, pady=2)
    global mainData
    mainData=[]
    mainData=[0]*257
    
    v = IntVar()
    v.set(1)  # initializing the choice, i.e. ASCII
    
#     print(mainData)
    def ReadMainMemory():
        global mainData,connection
        
        try:
            # READ_MEMORY_CARD
            SELECT = [0xFF, 0xB0,0x00,0b00000000,0xFF]
            addCommands("READ_MEMORY_CARD: ",toHexString(SELECT))

            data, sw1, sw2 = connection.transmit(SELECT)                
            addCommands("Respons: ","["+toHexString(data)+"] "+str(hex(sw1))+" "+str(hex(sw2)))
            for i in range(len(data)):
                mainData[i]=data[i]
    #         print("READ_MEMORY_CARD")
    #         print (mainData)
    #         print ("Command: %02X %02X" % (sw1, sw2))
            loadData(v.get())
        except SmartcardException as e:
            connectLb.config(fg="red",text=str(e).split('.')[1])
            addCommands("Error: ",str(e))

    btn_data_read = Button(card, text ="Read Main Memory", command = ReadMainMemory, padx=2, pady=2)
    btn_data_read.grid(row=0,column=1)
    
    def EditAll():
        
#         print("row",row)
        data = simpledialog.askstring("Input", "Enter data",parent=root)
#         print(data)
        if data!=None:
            memoryLocation=0            
            while memoryLocation<(len(data)) and memoryLocation<len(mainData):
                mainData[memoryLocation]=ord(data[memoryLocation])
                memoryLocation=memoryLocation+1
                
            loadData(v.get())
            
    btn_data_read = Button(card, text ="Edit All", command = EditAll, padx=2, pady=2)
    btn_data_read.grid(row=0,column=2)        
    #     print(mainData)
    def UpdateAll():
        # WRITE_MEMORY_CARD
        try:
            if auth==False:
                dataWriteLb.config(fg="red",text="Please Authentication First.")
                return
            answer = messagebox.askokcancel("Question","Do you want to update all memory location? ")
            if answer==True:
                SELECT = [0xFF, 0xD0, 0x00, 0b00000000,0xEF]
                in_data=mainData[:256]
                
                addCommands("WRITE_MEMORY_CARD ",toHexString(SELECT+in_data))

                data, sw1, sw2 = connection.transmit(SELECT+in_data)
                addCommands("Respons: ","["+toHexString(data)+"] "+str(hex(sw1))+" "+str(hex(sw2)))
                if sw1==144 and sw2==0:           
        #             messagebox.showinfo("Authentication", "authentication successful")
                    dataWriteLb.config(fg="green",text="Write successful ")
                    addCommands("WRITE_MEMORY_CARD: ","Write successful ")
                else:            
        #             messagebox.showinfo("authentication", "authentication failed")
                    dataWriteLb.config(fg="red",text="Write failed ")
                    addCommands("WRITE_MEMORY_CARD: ","Write failed ")
        except SmartcardException as e:
            connectLb.config(fg="red",text=str(e).split('.')[1])
            addCommands("Error: ",str(e))

    btn_data_read = Button(card, text ="Update All", command = UpdateAll, padx=2, pady=2)
    btn_data_read.grid(row=0,column=3)
   
    def ShowChoice():
        loadData(v.get())


    Radiobutton(card,text="ASCII",padx = 5, variable=v,command=ShowChoice,value=1).grid(row=0,column=4)
    Radiobutton(card,text="HEX",padx = 5, variable=v,command=ShowChoice,value=2).grid(row=0,column=5)
    
    
    card_left = Frame(card,  width=400, height=600)
    card_left.grid(row=1, column=0, padx=5, pady=5,sticky='ne')
    
    Label(card_left, text="Address 3 ").grid(row=0,column=0, padx=2, pady=2,sticky='ne')
    Label(card_left, text="Address 2 ").grid(row=1,column=0, padx=2, pady=2,sticky='ne')
    Label(card_left, text="Address 1 ").grid(row=2,column=0, padx=2, pady=2,sticky='ne')
    Label(card_left, text="Address 0 ").grid(row=3,column=0, padx=2, pady=2,sticky='ne')
    
    Label(card_left, text="Security Code 3 ").grid(row=0,column=1, padx=2, pady=2,sticky='ne')
    Label(card_left, text="Security Code 2 ").grid(row=1,column=1, padx=2, pady=2,sticky='ne')
    Label(card_left, text="Security Code 1 ").grid(row=2,column=1, padx=2, pady=2,sticky='ne')
    Label(card_left, text="Security Code 0 ").grid(row=3,column=1, padx=2, pady=2,sticky='ne')
    
    inAddr3=Entry(card_left, width=4)
    inAddr3.insert(0, "FF")
    inAddr3.grid(row=0,column=2, padx=2, pady=2,sticky='w')
    inAddr2=Entry(card_left, width=4)
    inAddr2.insert(0, "FF")
    inAddr2.grid(row=1,column=2, padx=2, pady=2,sticky='w')
    inAddr1=Entry(card_left, width=4)
    inAddr1.insert(0, "FF")
    inAddr1.grid(row=2,column=2, padx=2, pady=2,sticky='w')
    outAddr0=Entry(card_left, width=4)
    outAddr0.insert(0, "FF")
#     outAddr0.configure(state='readonly')
    outAddr0.grid(row=3,column=2, padx=2, pady=2,sticky='w')
      
    def Authenticate():
        global auth
        try:
            # Authentication
            SELECT = [0xFF, 0x20, 0x00, 0x00,0x03,int("0x"+inAddr1.get(),0),int("0x"+inAddr2.get(),0),int("0x"+inAddr3.get(),0)]
            
            addCommands("Authentication: ",toHexString(SELECT))
            
            data, sw1, sw2 = connection.transmit(SELECT)
            addCommands("Respons: ","["+toHexString(data)+"] "+str(hex(sw1))+" "+str(hex(sw2)))
    #         print(sw2)
            outAddr0.delete(0,END)
            outAddr0.insert(0, sw2)
            if sw1==144 and (sw2==7 or sw2==255):
                auth=True
    #             messagebox.showinfo("Authentication", "authentication successful")
                autLb.config(fg="green",text="Authentication successful ")
                addCommands("Authentication: ","Authentication successful ")
            else:            
    #             messagebox.showinfo("authentication", "authentication failed")
                autLb.config(fg="red",text="Authentication failed ")
                addCommands("Authentication: ","Authentication failed ")
            
        except SmartcardException as e:
            connectLb.config(fg="red",text=str(e).split('.')[1])
            addCommands("Error: ",str(e))
            
    btn_auth = Button(card_left, text ="Authenticate", command = Authenticate, padx=2, pady=2)
    btn_auth.grid(row=4, column=0,sticky='w')
    
    
    def ChangeCode():
        try:
            answer = messagebox.askokcancel("Question","Do you want to update secret code? ")
            if answer == True:
                # Authentication
                SELECT = [0xFF, 0xD2, 0x00, 0x01,0x03,int("0x"+inAddr1.get(),0),int("0x"+inAddr2.get(),0),int("0x"+inAddr3.get(),0)]
                
                addCommands("Secret Code Change: ",toHexString(SELECT))
                
                data, sw1, sw2 = connection.transmit(SELECT)
                addCommands("Respons: ","["+toHexString(data)+"] "+str(hex(sw1))+" "+str(hex(sw2)))
    #             print(sw2)
                outAddr0.delete(0,END)
                outAddr0.insert(0, sw2)
                if sw1==144 and sw2==0:           
        #             messagebox.showinfo("Authentication", "authentication successful")
                    autLb.config(fg="green",text="Code Change successful ")
                    addCommands("Secret Code Change: ","Code Change successful ")
                else:            
        #             messagebox.showinfo("authentication", "authentication failed")
                    autLb.config(fg="red",text="Code Change failed ")
                    addCommands("Secret Code Change: ","Code Change failed ")
        except SmartcardException as e:
            connectLb.config(fg="red",text=str(e).split('.')[1])
            addCommands("Error: ",str(e))

    btn_change_code = Button(card_left, text ="Change Code", command = ChangeCode, padx=2, pady=2)
    btn_change_code.grid(row=4, column=1,sticky='w')
    
    
    
    
    autLb=Label(card_left, text="Unauthorized ",fg="red")
    autLb.grid(row=5,column=0,columnspan=2, padx=2, pady=2,sticky='ne')
    
    dataWriteLb=Label(card_left, text=" ",fg="red")
    dataWriteLb.grid(row=6,column=0,columnspan=2, padx=2, pady=2,sticky='ne')
    
    card_right = Frame(card,  width=400, height=600,bg='white')
    card_right.grid(row=1, column=1,columnspan=5, padx=5, pady=5,sticky='ne')
        
    def Write(row):
        try:
            if auth==False:
                dataWriteLb.config(fg="red",text="Please Authentication First.")
                return
            # WRITE_MEMORY_CARD
            memoryLocation=16*int(row)
            answer = messagebox.askokcancel("Question","Do you want to update memory location "+hex(memoryLocation)+" to " +hex(memoryLocation+16))
            if answer==True:
                SELECT = [0xFF, 0xD0, 0x00, memoryLocation,0x1F]
                in_data=mainData[memoryLocation:memoryLocation+16]
                
                addCommands("WRITE_MEMORY_CARD: ",toHexString(SELECT+in_data))

                data, sw1, sw2 = connection.transmit(SELECT+in_data)
                addCommands("Respons: ","["+toHexString(data)+"] "+str(hex(sw1))+" "+str(hex(sw2)))
                if sw1==144 and sw2==0:           
        #             messagebox.showinfo("Authentication", "authentication successful")
                    dataWriteLb.config(fg="green",text="Write successful ")
                    addCommands("WRITE_MEMORY_CARD: ","Write successful ")
                else:            
        #             messagebox.showinfo("authentication", "authentication failed")
                    dataWriteLb.config(fg="red",text="Write failed ")
                    addCommands("WRITE_MEMORY_CARD: ","Write failed ")
        except SmartcardException as e:
            connectLb.config(fg="red",text=str(e).split('.')[1])
            addCommands("Error: ",str(e))
            
    def Edit(row):
        
#         print("row",row)
        data = simpledialog.askstring("Input", "Enter data",parent=root)
#         print(data)
        if data!=None:
            memoryLocation=16*int(row)
            for i in range(len(data)):
                mainData[memoryLocation]=ord(data[i])
                memoryLocation=memoryLocation+1
            loadData(v.get())
            
        
    def loadData(viewType):
        lst=card_right.grid_slaves()
        for l in lst:
            l.destroy()
#         card_right.destroy()
        height = 16
        width = 16
#         card_right.grid_forget()
        Label(card_right, text="Address").grid(row=0, column=0, padx=2, pady=2,sticky='ne')
        for i in range(width):
            Label(card_right, text="+"+str(i)).grid(row=0, column=i+1, padx=2, pady=2,sticky='ne')

        for i in range(height): #Rows
            Label(card_right, text=str(16*i)+"-"+str(16*(i+1)-1)).grid(row=i+1, column=0, padx=2, pady=2,sticky='ne')
            for j in range(width): #Columns
                if viewType==1:
                    Label(card_right, text=chr(mainData[i*16+j])).grid(row=i+1, column=j+1, padx=2, pady=2)
                else:
                    Label(card_right, text=hex(mainData[i*16+j])).grid(row=i+1, column=j+1, padx=2, pady=2)


            
            Button(card_right, text ="Write", command = lambda i=i: Write(str(i)), padx=2, pady=2).grid(row=i+1, column=17,sticky='w')
            Button(card_right, text ="Edit", command = lambda i=i: Edit(str(i)), padx=2, pady=2).grid(row=i+1, column=18,sticky='w')


    loadData(v.get())
#     Process = threading.Thread(target=findCardReader)
#     Process.start()
    
    root.mainloop()
#     root.destroy()
    
#     Process.join()
    

if __name__ == "__main__":
    gui()